# SimPy

`simpy` uses process based model worldview.  Given its simplicity it is a highly flexible discrete-event simulation package.

> Detailed documentation can be found here: https://simpy.readthedocs.io/en/latest/

---

## Imports

It is recommended that you use the provided conda virtual environment `os-sim`.  

>If you are running this code in **Google Colab** then `simpy` can be installed by:

In [ ]:
pip install simpy==4.0.1

In [2]:
import simpy
simpy.__version__

'4.0.1'

In [3]:
import numpy as np
import pandas as pd
import itertools
import math
import matplotlib.pyplot as plt

In [4]:
# need numpy > v1.18
np.__version__

'1.19.2'

# Example: A health clinic in the US.

This example is based on Nelson (2010) ex. 13 page 170.

In [6]:
TRACE = True

In [5]:
def trace(msg):
    '''
    Print a simulated trace.
    '''
    if TRACE:
        print(msg)

In [ ]:
class TraumaPathway(object):
    '''
    Encapsulates the process a patient with severe injuries or illness.
    
    These patients are signed into the ED and triaged as having severe injuries
    or illness.
    
    Patients are stabilised in resus (trauma) and then sent to Treatment.  
    Following treatment they are discharged.
    '''
    def __init__(self, identifier, env, args):
        '''
        Constructor method
        
        Params:
        -----
        identifier: int
            a numeric identifier for the patient.
            
        env: simpy.Environment
            the simulation environment
            
        args: Scenario
            Container class for the simulation parameters
            
        '''
        self.identifier = identifier
        self.env = env
        
        # triage resource
        self.triage = args.triage
        
        #waiting time metrics
        self.wait_triage = -np.inf
        self.wait_trauma = -np.inf
        
         
    def service(self):
        '''
        simulates the major treatment process for a patient
        
        1. request and wait for sign-in/triage
        2. trauma
        3. treatment
        '''
        # record the time that call entered the queue
        start_wait = env.now

        # request sign-in/triage 
        with self.triage.request() as req:
            yield req
            
            # record the waiting time for call to be answered
            self.waiting_time = self.env.now - start_wait
            trace(f'operator answered call {self.identifier} at '
                  f'{self.env.now:.3f}')
            
            # sample call duration.
            triage_duration = args.triage_dist.sample()
            yield self.env.timeout(triage_duration)
            
            trace(f'triage {self.identifier} ended {self.env.now:.3f}; '
                  f'waiting time was {self.waiting_time:.3f}')
            
        # request trauma room 
        with self.args.trauma.request() as req:
            yield req
            
            # record the waiting time for call to be answered
            self.waiting_time = self.env.now - start_wait
            trace(f'operator answered call {self.identifier} at '
                  f'{self.env.now:.3f}')
            
            # sample call duration.
            triage_duration = args.triage_dist.sample()
            yield self.env.timeout(triage_duration)
            
            trace(f'triage {self.identifier} ended {self.env.now:.3f}; '
                  f'waiting time was {self.waiting_time:.3f}')
            
        